# 09 - Embedding + Vector Stores 向量化

## 用途
学习使用 DashScope Embedding 将文本转换为向量，并用 FAISS 进行向量存储和相似度检索

## 学习目标
- 掌握 DashScope Embeddings 使用方法
- 理解文本向量化原理和过程
- 能进行中文文本向量化
- 掌握向量库创建和相似度检索

## 🔑 API 配置要求
**注意**：需要配置 DashScope API 密钥，与 OpenAI 分开计费

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. DashScope Embeddings 基础使用

In [ ]:
# DashScope Embeddings 基础使用 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🔤 DashScope Embeddings 基础测试:")
print("=" * 50)

# 检查 DashScope API 配置
dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
if not dashscope_api_key:
    print("❌ 未找到 DASHSCOPE_API_KEY 环境变量")
    print("   请在 .env 文件中添加: DASHSCOPE_API_KEY=your_key_here")
    print("   或设置环境变量: export DASHSCOPE_API_KEY=your_key_here")
else:
    print(f"✅ DashScope API Key 已配置 (长度: {len(dashscope_api_key)})")
    
    try:
        # 初始化 DashScope Embeddings
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",  # 使用基础模型控制成本
            dashscope_api_key=dashscope_api_key
        )
        
        print(f"\n📝 Embeddings 配置:")
        print(f"   模型: text-embedding-v1")
        print(f"   维度: 1536 (预期)")
        
        # 测试单句中文向量化
        test_text = "LangChain 是一个强大的 LLM 应用开发框架"
        print(f"\n🧪 测试文本: {test_text}")
        
        # 生成向量
        embedding_vector = embeddings.embed_query(test_text)
        
        print(f"\n📊 向量化结果:")
        print(f"   向量类型: {type(embedding_vector)}")
        print(f"   向量维度: {len(embedding_vector)}")
        print(f"   数据类型: {type(embedding_vector[0])}")
        print(f"   数值范围: [{min(embedding_vector):.4f}, {max(embedding_vector):.4f}]")
        print(f"   向量范数: {sum(x**2 for x in embedding_vector)**0.5:.4f}")
        
        # 显示向量前10个值
        print(f"\n🔢 向量前10个值:")
        print(f"   {embedding_vector[:10]}")
        
        # 验证点：能成功生成数值向量
        assert isinstance(embedding_vector, list), "向量不是列表类型"
        assert len(embedding_vector) > 0, "向量为空"
        assert all(isinstance(x, (int, float)) for x in embedding_vector), "向量包含非数值"
        print(f"\n✅ 验证通过：DashScope 向量化正常工作")
        
    except Exception as e:
        print(f"\n❌ DashScope API 调用失败: {e}")
        print("   可能原因:")
        print("   1. API Key 无效或过期")
        print("   2. 网络连接问题")
        print("   3. API 配额不足")
        print("   4. 模型名称错误")

## 2. 批量文本向量化测试

In [ ]:
# 批量文本向量化测试 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from dotenv import load_dotenv
import os
import time

# 加载环境变量
load_dotenv()

print("📦 批量文本向量化测试:")
print("=" * 50)

dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
if not dashscope_api_key:
    print("❌ DashScope API Key 未配置")
else:
    try:
        # 初始化 Embeddings
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        # 准备测试文本（控制数量以节省成本）
        test_texts = [
            "LangChain 是一个用于构建 LLM 应用的框架",
            "Python 是一种流行的编程语言",
            "机器学习是人工智能的重要分支",
            "向量化是文本处理的关键技术",
            "FAISS 是高效的向量检索库"
        ]
        
        print(f"📝 测试文本数量: {len(test_texts)}")
        print(f"💰 成本控制: 使用最小测试集")
        
        # 单个向量化测试
        print(f"\n🔍 单个向量化测试:")
        start_time = time.time()
        single_vector = embeddings.embed_query(test_texts[0])
        single_time = time.time() - start_time
        
        print(f"   处理时间: {single_time:.2f} 秒")
        print(f"   向量维度: {len(single_vector)}")
        
        # 批量向量化测试
        print(f"\n📦 批量向量化测试:")
        start_time = time.time()
        batch_vectors = embeddings.embed_documents(test_texts)
        batch_time = time.time() - start_time
        
        print(f"   处理时间: {batch_time:.2f} 秒")
        print(f"   平均每条: {batch_time/len(test_texts):.2f} 秒")
        print(f"   向量数量: {len(batch_vectors)}")
        
        # 分析批量结果
        print(f"\n📊 批量结果分析:")
        for i, (text, vector) in enumerate(zip(test_texts, batch_vectors), 1):
            print(f"   文本 {i}: {len(vector)} 维")
            print(f"   内容: {text[:30]}...")
        
        # 检查向量一致性
        dimensions = [len(vector) for vector in batch_vectors]
        if len(set(dimensions)) == 1:
            print(f"\n✅ 向量维度一致: {dimensions[0]} 维")
        else:
            print(f"\n⚠️  向量维度不一致: {dimensions}")
        
        # 性能对比
        print(f"\n⚡ 性能对比:")
        print(f"   单条处理: {single_time:.2f} 秒")
        print(f"   批量处理: {batch_time:.2f} 秒")
        print(f"   效率提升: {(single_time * len(test_texts) / batch_time):.1f}x")
        
        # 验证点：批量向量化正确
        assert len(batch_vectors) == len(test_texts), "向量数量不匹配"
        assert all(len(vector) > 0 for vector in batch_vectors), "存在空向量"
        print(f"\n✅ 验证通过：批量向量化正常工作")
        
    except Exception as e:
        print(f"\n❌ 批量向量化失败: {e}")

## 3. FAISS 向量库创建和存储

In [ ]:
# FAISS 向量库创建和存储 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
import numpy as np

# 加载环境变量
load_dotenv()

print("🗄️  FAISS 向量库创建和存储测试:")
print("=" * 50)

dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
if not dashscope_api_key:
    print("❌ DashScope API Key 未配置")
else:
    try:
        # 初始化 Embeddings
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        # 准备文档数据（控制数量以节省成本）
        documents = [
            "LangChain 是一个开源框架，用于简化 LLM 应用开发",
            "Python 是一种高级编程语言，语法简洁易学",
            "机器学习通过数据训练模型，实现智能决策",
            "向量化将文本转换为数值向量，便于数学计算",
            "FAISS 是 Facebook 开发的高效向量检索库"
        ]
        
        metadatas = [
            {"source": "langchain_doc", "category": "framework"},
            {"source": "python_doc", "category": "language"},
            {"source": "ml_doc", "category": "ai"},
            {"source": "embedding_doc", "category": "technique"},
            {"source": "faiss_doc", "category": "library"}
        ]
        
        print(f"📝 准备文档数量: {len(documents)}")
        print(f"💰 成本控制: 最小测试集")
        
        # 创建 FAISS 向量库
        print(f"\n🏗️  创建 FAISS 向量库:")
        vector_store = FAISS.from_texts(
            texts=documents,
            embedding=embeddings,
            metadatas=metadatas
        )
        
        print(f"   向量库类型: {type(vector_store)}")
        print(f"   文档数量: {vector_store.index.ntotal}")
        print(f"   向量维度: {vector_store.index.d}")
        
        # 测试向量库基本信息
        print(f"\n📊 向量库信息:")
        print(f"   索引类型: {type(vector_store.index).__name__}")
        print(f"   是否可训练: {vector_store.index.is_trained}")
        
        # 测试添加单个文档
        print(f"\n➕ 添加单个文档:")
        new_doc = "向量相似度计算使用余弦相似度或欧氏距离"
        new_meta = {"source": "similarity_doc", "category": "technique"}
        
        vector_store.add_texts([new_doc], [new_meta])
        print(f"   添加后文档数: {vector_store.index.ntotal}")
        
        # 验证点：向量库创建成功
        assert vector_store.index.ntotal > 0, "向量库为空"
        assert vector_store.index.d > 0, "向量维度为0"
        print(f"\n✅ 验证通过：FAISS 向量库创建成功")
        
        # 保存向量库到本地（可选）
        print(f"\n💾 保存向量库到本地:")
        save_path = "./faiss_index"
        vector_store.save_local(save_path)
        print(f"   保存路径: {save_path}")
        print(f"   文件数量: {len(os.listdir(save_path))}")
        
        # 清理保存的文件
        import shutil
        if os.path.exists(save_path):
            shutil.rmtree(save_path)
            print(f"   🧹 清理临时文件")
        
    except Exception as e:
        print(f"\n❌ FAISS 向量库创建失败: {e}")
        print("   可能原因:")
        print("   1. FAISS 库未安装")
        print("   2. 向量化失败")
        print("   3. 内存不足")

## 4. 相似度检索测试

In [ ]:
# 相似度检索测试 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🔍 相似度检索测试:")
print("=" * 50)

dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
if not dashscope_api_key:
    print("❌ DashScope API Key 未配置")
else:
    try:
        # 初始化 Embeddings
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        # 准备测试数据
        documents = [
            "LangChain 是一个开源框架，用于简化 LLM 应用开发",
            "Python 是一种高级编程语言，语法简洁易学",
            "机器学习通过数据训练模型，实现智能决策",
            "向量化将文本转换为数值向量，便于数学计算",
            "FAISS 是 Facebook 开发的高效向量检索库"
        ]
        
        metadatas = [
            {"source": "langchain_doc", "category": "framework"},
            {"source": "python_doc", "category": "language"},
            {"source": "ml_doc", "category": "ai"},
            {"source": "embedding_doc", "category": "technique"},
            {"source": "faiss_doc", "category": "library"}
        ]
        
        # 创建向量库
        vector_store = FAISS.from_texts(documents, embeddings, metadatas)
        
        print(f"📚 向量库创建完成: {vector_store.index.ntotal} 个文档")
        
        # 测试相似度检索
        test_queries = [
            "如何开发人工智能应用",
            "编程语言的特点",
            "数据检索技术"
        ]
        
        for i, query in enumerate(test_queries, 1):
            print(f"\n🔍 查询 {i}: {query}")
            
            # 执行相似度搜索
            results = vector_store.similarity_search_with_score(query, k=3)
            
            print(f"   检索结果数量: {len(results)}")
            
            for j, (doc, score) in enumerate(results, 1):
                print(f"   结果 {j} (相似度: {score:.4f}):")
                print(f"     内容: {doc.page_content}")
                print(f"     来源: {doc.metadata}")
        
        # 测试按阈值检索
        print(f"\n🎯 按阈值检索测试:")
        threshold_query = "机器学习算法"
        threshold_results = vector_store.similarity_search_with_score(
            threshold_query, 
            k=5,
            score_threshold=0.3  # 相似度阈值
        )
        
        print(f"   查询: {threshold_query}")
        print(f"   阈值: 0.3")
        print(f"   结果数量: {len(threshold_results)}")
        
        for doc, score in threshold_results:
            print(f"   - {doc.page_content} (相似度: {score:.4f})")
        
        # 测试最大边际相关性检索（MMR）
        print(f"\n🔄 MMR 检索测试:")
        mmr_results = vector_store.max_marginal_relevance_search(
            "编程和人工智能", 
            k=3,
            fetch_k=5
        )
        
        print(f"   查询: 编程和人工智能")
        print(f"   MMR 结果数量: {len(mmr_results)}")
        
        for i, doc in enumerate(mmr_results, 1):
            print(f"   结果 {i}: {doc.page_content}")
        
        # 验证点：能返回最相似的中文文本片段
        assert len(results) > 0, "检索结果为空"
        assert all(hasattr(doc, 'page_content') for doc, _ in results), "结果缺少内容"
        print(f"\n✅ 验证通过：相似度检索正常工作")
        
    except Exception as e:
        print(f"\n❌ 相似度检索失败: {e}")

## 5. 学习总结与成本控制

In [ ]:
# 学习总结与成本控制 - 独立代码块
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📋 Embedding + Vector Stores 学习总结:")
print("=" * 50)

# 验证点检查
verification_points = [
    "✅ DashScope Embeddings：文本向量化基础",
    "✅ 批量向量化：多文本处理优化",
    "✅ FAISS 向量库：存储和管理",
    "✅ 相似度检索：多种检索策略",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 Embedding + Vector Stores 最佳实践:")
print("1. API 管理：分别配置 OpenAI 和 DashScope 密钥")
print("2. 成本控制：使用最小测试集，避免批量处理")
print("3. 模型选择：text-embedding-v1 平衡性能和成本")
print("4. 向量库：FAISS 适合本地快速检索")
print("5. 检索策略：根据需求选择相似度或 MMR")

print("\n💰 成本控制策略:")
print("1. 测试阶段：使用 3-5 条短文本")
print("2. 缓存向量：避免重复向量化相同内容")
print("3. 批量处理：一次处理多条比单条更高效")
print("4. 本地存储：保存向量库避免重复计算")
print("5. 监控使用：定期检查 API 调用统计")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 Retrievers 检索器配置")
print("2. 掌握 RAG 系统完整流程")
print("3. 学习其他向量库（Chroma、Pinecone）")
print("4. 探索向量检索优化算法")
print("5. 实践生产级 RAG 应用开发")

# 最终验证：确保基础功能可用
try:
    dashscope_api_key = os.getenv('DASHSCOPE_API_KEY')
    
    if dashscope_api_key:
        # 简单测试
        embeddings = DashScopeEmbeddings(
            model="text-embedding-v1",
            dashscope_api_key=dashscope_api_key
        )
        
        test_vector = embeddings.embed_query("测试文本")
        
        print(f"\n🎉 最终验证成功:")
        print(f"   API 状态: 可用")
        print(f"   向量维度: {len(test_vector)}")
        print(f"   向量类型: {type(test_vector)}")
        print("\n✅ Embedding + Vector Stores 学习完成！")
        
        print(f"\n⚠️  重要提醒:")
        print("   - DashScope API 与 OpenAI 分开计费")
        print("   - 生产环境请设置合理的调用限制")
        print("   - 建议使用向量缓存减少 API 调用")
        
    else:
        print(f"\n⚠️  最终验证跳过: DashScope API Key 未配置")
        print("   请配置 DASHSCOPE_API_KEY 后重试")
        
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")